In [1]:
import pandas as pd

train_df = pd.read_csv("./titanic/train.csv")
test_df = pd.read_csv("./titanic/test.csv")

AverageAge = train_df.Age.mean()
train_df.Age = train_df.Age.fillna(AverageAge)

AverageAge = test_df.Age.mean()
test_df.Age = test_df.Age.fillna(AverageAge)

AverageFare = test_df.Fare.mean()
test_df.Fare = test_df.Fare.fillna(AverageFare)

FrequentEmbarked = train_df.Embarked.value_counts().idxmax()
train_df.Embarked = train_df.Embarked.fillna(FrequentEmbarked)

column_to_use = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
train_x = train_df[column_to_use].copy()
test_x = test_df[column_to_use].copy()

train_x["Sex"] = train_x.Sex.apply(lambda x: 0 if x=="male" else 1)
test_x["Sex"] = test_x.Sex.apply(lambda x: 0 if x=="male" else 1)

Embark2Float = {"S":0, "C":1, "Q":2}
train_x.Embarked = train_x.Embarked.apply(lambda x:Embark2Float[x])
test_x.Embarked = test_x.Embarked.apply(lambda x:Embark2Float[x])

from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
train_x = ss.fit_transform(train_x)
test_x = ss.transform(test_x)

In [2]:
from sklearn.model_selection import train_test_split

train_y = train_df.Survived
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2)

train_y = train_y.to_numpy()
val_y = val_y.to_numpy()

In [3]:
from LR import LogisticRegression
import time

lr = LogisticRegression(max_iter=500, penalty="l2", initialize = "LeCun")
start = time.time()
coef, interrupt = lr.fit(train_x, train_y)
print(f'time: {time.time() - start}')
print(f"Accuray: {lr.score(val_x, val_y):.3f}")

time: 0.07483196258544922
Accuray: 0.827


In [4]:
lr.feature_importance(coef, column_to_use)

rank : #1, feature : Sex, weight : 1.9866510238255959
rank : #2, feature : Age, weight : -1.2447335699233002
rank : #3, feature : SibSp, weight : -0.4540855230128705
rank : #4, feature : Embarked, weight : 0.3293972307411424
rank : #5, feature : Pclass, weight : -0.32278862760068483
rank : #6, feature : Parch, weight : 0.06773744739821774
rank : #7, feature : Fare, weight : 0.02776125286372133


In [5]:
import numpy as np
np.sign(lr.w)

array([-1.,  1., -1., -1.,  1.,  1.,  1.])